In [1]:
!pip install pandas scikit-learn matplotlib seaborn plotly folium

In [ ]:
import pandas as pd

df = pd.read_csv('traffic_density_202501.csv')

df['Tarih_Saat'] = pd.to_datetime(df['DATE_TIME'])

df['Saat'] = df['Tarih_Saat'].dt.hour
df['Gun_Adi'] = df['Tarih_Saat'].dt.day_name()
df['Hafta_Ici_Sonu'] = df['Tarih_Saat'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0) # 0: Hafta İçi, 1: Hafta Sonu

print("Eksik değerler:\n", df.isnull().sum())
df.dropna(inplace=True)

features = ['LATITUDE', 'LONGITUDE', 'AVERAGE_SPEED', 'MINIMUM_SPEED', 'MAXIMUM_SPEED', 'NUMBER_OF_VEHICLES', 'Saat', 'Hafta_Ici_Sonu']


X = df[features]

print("\nSeçilen özellikler ile DataFrame'in ilk 5 satırı:")
print(X.head())

In [ ]:
from sklearn.preprocessing import StandardScaler

# StandardScaler kullanarak veriyi ölçekleme
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Ölçeklenmiş veriyi DataFrame'e geri çevirme
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

print("Ölçeklenmiş verinin ilk 5 satırı:")
print(X_scaled_df.head())

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Veri setinin boyutunu kontrol edelim
print(f"X_reduced veri setinin boyutu (satır, sütun): {X_reduced.shape}")


sample_size = min(len(X_reduced), 100000) # Maksimum 100.000 satır veya mevcut tüm veri
if len(X_reduced) > sample_size:
    print(f"Elbow metodu için veri setinden {sample_size} satırlık bir örneklem alınıyor.")

    random_indices = np.random.choice(X_reduced.shape[0], sample_size, replace=False)
    X_reduced_sampled = X_reduced[random_indices]
else:
    X_reduced_sampled = X_reduced
    print("Elbow metodu için tüm veri seti kullanılıyor (zaten yeterince küçük).")



wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(X_reduced_sampled)
    wcss.append(kmeans.inertia_)

# Elbow metodunu görselleştirme
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('Elbow Metodu ile Optimal Küme Sayısı (Örneklenmiş Veri Üzerinde)')
plt.xlabel('Küme Sayısı (k)')
plt.ylabel('WCSS (Küme İçi Kareler Toplamı)')
plt.grid(True)
plt.show()

optimal_k = 4


print(f"\nOptimal k={optimal_k} değeri ile K-Means tüm X_reduced veri seti üzerinde çalıştırılıyor...")
kmeans = KMeans(n_clusters=optimal_k, init='k-means++', max_iter=300, n_init=10, random_state=42)
clusters = kmeans.fit_predict(X_reduced)

df['Cluster'] = clusters

print(f"\nVeri setine {optimal_k} küme atanmıştır.")
print("Her kümedeki veri sayısı:\n", df['Cluster'].value_counts())

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

if X_reduced.shape[1] > 2:
    pca_viz = PCA(n_components=2, random_state=42)
    principal_components = pca_viz.fit_transform(X_reduced)
else: # Eğer X_reduced zaten 2 boyutlu ise
    principal_components = X_reduced

# İndirgenmiş boyutları bir DataFrame'e dönüştürme
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['Cluster'] = df['Cluster'] # Küme etiketlerini ekleme

# PCA sonuçlarını kümelere göre görselleştirme
plt.figure(figsize=(12, 8))
sns.scatterplot(x='PC1', y='PC2', hue='Cluster', data=pca_df,
                palette='viridis', s=100, alpha=0.7)
plt.title('PCA ile Boyut İndirgenmiş Verinin Kümelere Göre Dağılımı')
plt.xlabel('Birincil Bileşen 1 (PC1)')
plt.ylabel('Birincil Bileşen 2 (PC2)')
plt.grid(True)
plt.legend(title='Küme')
plt.show()

In [ ]:
import folium

# İstanbul'un ortalama koordinatları (haritanın merkezini belirlemek için)
istanbul_lat = df['LATITUDE'].mean()
istanbul_lon = df['LONGITUDE'].mean()

# Harita oluşturma
m = folium.Map(location=[istanbul_lat, istanbul_lon], zoom_start=10)


colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen',
          'cadetblue', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray', 'darkpurple', 'white']

sample_df_for_map = df.sample(n=min(len(df), 100000), random_state=42) # Max 100k nokta göster

for index, row in sample_df_for_map.iterrows():
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=3, # Nokta boyutu
        color=colors[row['Cluster'] % len(colors)], # Küme numarasına göre renk
        fill=True,
        fill_color=colors[row['Cluster'] % len(colors)],
        fill_opacity=0.7,
        # Araç sayısı ve Ortalama Hız bilgilerini tooltip olarak ekledim
        tooltip=f"Küme: {row['Cluster']}<br>Araç Sayısı: {row['NUMBER_OF_VEHICLES']}<br>Ortalama Hız: {row['AVERAGE_SPEED']}"
    ).add_to(m)

# Haritayı Colab çıktısında gösterme
m

In [ ]:
cluster_means = df.groupby('Cluster')[['LATITUDE', 'LONGITUDE', 'AVERAGE_SPEED', 'MINIMUM_SPEED', 'MAXIMUM_SPEED', 'NUMBER_OF_VEHICLES', 'Saat', 'Hafta_Ici_Sonu']].mean()
print("\nHer kümenin ortalama özellikleri:\n", cluster_means)